# Title: Effective Resistance Calculations with Julia
## Author: Joe Malone

## Introduction:
Using effective resistance to calculate the connectivity of a graph is a [commonly studied topic](https://www.nas.ewi.tudelft.nl/people/Piet/papers/LAA_2011_EffectiveResistance.pdf) in graph theory. It's applications can be used in recommender systems or any research that desires to know how closely related two nodes of a graph are. One of the main focuses of research in effective resistance relies on calculating effective resistance quickly and efficiently. This amounts to solving a Laplacian system of equations quickly and efficiently. Daniel Spielman and Shang Hua-Teng have been able to break through what was the previously supposed limit for run time of effective resistance calculators. They developed a [nearly-linear time solver](https://arxiv.org/abs/cs/0310051) for linear systems based off of previous work in graph theory. Daniel Spielman has written a library that includes this solver among others in the Laplacians.jl library for Julia.

## Libraries Used:
* [IPython](https://ipython.org/)
* [Julia](https://julialang.org/)
* [Laplacians](https://github.com/danspielman/Laplacians.jl)

## Abstract:

This project will examine the efficacy of the Laplacians library in Julia to calculate effective resistance between nodes in large networks. It will explain the usage of the library and show how to run through the calculations on a simple example at first and will then generalize the methods to more complicated networks. The project will mine data from the citeulike dataset for larger networks and then compare the nearly-linear time system solver to the exact solver to see which is quicker and more accurate. The last part of the project will allow readers to try creating different graphs and using the solver to calculate effective resistance between nodes. The project is mostly focused on displaying some of the capabilities of Julia and the Laplacians library.

* [Github Repository](https://github.com/joe-malone/MATH-725-final.git) for This Project
* Note that to run the code, you will need to first run `Pkg.add("Laplacians")`

In [1]:
using Laplacians

## Run through simple example to make sure the solver is working as we expect
Note: This example is based off of a homework problem in MATH 725 at K-State  

The below code creates the adjacency matrix for a simple undirected graph, sparsifies the adjacency matrix, and creates a linear system solver with the function from Laplacians.jl based off of Cholesky factorization of the Laplacian.

In [2]:
# Test the solver with a known solution
# V = { 1, 2, 3, 4, 5 }
# E = {[1,2], [1,4], [2,3], [2,4], [3,4], [3,5], [4,5]}

num_nodes = 5

A = [0 1 0 1 0;
     1 0 1 1 0;
     0 1 0 1 1;
     1 1 1 0 1;
     0 0 1 1 0]

simpleSparseAdj = sparse(A)
solver = cholLap(simpleSparseAdj; verbose=true)

Solver build time: 0.323 seconds.
Solver build time: 1.914 seconds.


(::#100) (generic function with 1 method)

Since we set the verbose parameter to true, Julia will print out the time to build the solver and the time to solve a system of equations. The vector for we create has nonzero entries at the places of node 1 and node 5. The solver returns a vector of the relative resistances. By taking the difference of the entry at location 5 and the entry at location 1, we see the effective resistance between nodes 1 and 5. Note here that Julia has 1-indexed arrays, so the first position is 1, not 0 as in Python.

In [3]:
# Figure out how the solver works with a problem where we know the answer
vec = [-1.0 0 0 0 1]   # Vector to compute effective resistance from node 1 to node 5
soln = solver(vec)
result = soln[5] - soln[1]
println(result)
println(result*7)

Solve time: 0.017 seconds.
1.1428571428571426
7.999999999999998


Next we will create a function to compute the effective resistance between two nodes more concisely. We first create a sparse vector so that we only need to initialize the nonzero entries. In Julia, it is much faster to create a full vector by calling full(vec) than going through and initializing all of the entries.

In [13]:
# Computes the effective resistance with the given solver between 2 nodes. Also pass in the number of nodes in the graph.
function compute_effective_resistance(solver, node_1, node_2, num_nodes)
    vec = sparsevec(Dict(node_1=>-1.0, node_2=>1.0), num_nodes)
    soln = solver(full(vec))
    soln[node_2] - soln[node_1]
end

compute_effective_resistance (generic function with 1 method)

The following code will calculate the effective resistances in a matrix. The entry at x, y is the effective resistance between node x and node y.

In [5]:
# Set up effective resistance matrix to hold all of the values
R_effs = zeros(num_nodes, num_nodes)

# Loop through to compute the upper values (diagonals will all be 0, lower will be reflection of upper)
for x = 1:num_nodes-1
    R_effs[x, x+1:num_nodes] = [compute_effective_resistance(solver, x, y, num_nodes) for y=x+1:num_nodes]
end

Symmetric(R_effs)

Solve time: 0.0 seconds.
Solve time: 0.0 seconds.
Solve time: 0.0 seconds.
Solve time: 0.0 seconds.
Solve time: 0.0 seconds.
Solve time: 0.0 seconds.
Solve time: 0.0 seconds.
Solve time: 0.0 seconds.
Solve time: 0.0 seconds.
Solve time: 0.0 seconds.


5×5 Symmetric{Float64,Array{Float64,2}}:
 0.0       0.619048  0.904762  0.619048  1.14286 
 0.619048  0.0       0.571429  0.47619   0.904762
 0.904762  0.571429  0.0       0.47619   0.619048
 0.619048  0.47619   0.47619   0.0       0.619048
 1.14286   0.904762  0.619048  0.619048  0.0     

# Take what we have learned in the simple example and apply it to our weighted graph (the more complex case)

This code reads from the citeulike database and closes the connection to the file.

In [6]:
file_path = "ctrsr_datasets/citeulike-a/users.dat";
num_users = 5551;     # from README of dataset
num_papers = 16980;   # from README of dataset

In [7]:
f = open(file_path, "r");
lines = readlines(f);
close(f);

### Code to iterate through the lines of the file and print content
```
counter = 1
for l in lines
    println("$counter $l")
    counter += 1
end
```

The `@time` macro shows the time it takes to run a command. In this case, we create an array of Sets. Each Set contains all of the citations from a particular user (based on the array position). We will use this to create a weighted adjacency matrix.

In [8]:
# Create the array of user-citations (each index represents a user, each element of a set is a citation)
@time user_citations = [Set{String}(deleteat!(split(l," "), 1)) for l in lines];

  0.223801 seconds (514.75 k allocations: 31.170 MiB, 5.77% gc time)


### Define the weight $\omega$ of each edge as the following:
If $|P_a| = 0$ or $|P_b| = 0$, then $\omega = 0$
otherwise, $\omega$ = mean(s/$|P_a|$, s/$|P_b|$)    

where $\omega$ is the weight, $|P_a|$ is the number of papers node a has cited, $|P_b|$ is the number of papers node b has cited, and s is the number of papers that they have in common.  

The following code creates a sparse adjacency matrix of all of the weights between users.

In [9]:
# Note that the adjacency matrix will be symmetrical in this case, and all weights will be between 0 and 1
# where 0 means that there is no citations in common and 1 means that all of the citations are in common
Arr = Array{Float64, 2}(num_users, num_users);

# Use indexes to initialize the array
for user_a = 1:num_users
    P_a = user_citations[user_a]
    for user_b = user_a+1:num_users
        # Since all array values initally zero, we only need to initialize the nonzero values
        P_b = user_citations[user_b]
        if length(intersect(P_a, P_b)) > 0
            s = length(intersect(P_a, P_b))
            Arr[user_a, user_b] = 1 / mean([s/length(P_a), s/length(P_b)])
        end
    end
end

# Use the array we have created to make a sparse adjacency matrix of the weighted graph
sparseAdj = sparse(Symmetric(Arr))

5551×5551 SparseMatrixCSC{Float64,Int64} with 2219312 stored entries:
  [8   ,    1]  =  8.75
  [18  ,    1]  =  24.7059
  [47  ,    1]  =  85.8564
  [56  ,    1]  =  75.9477
  [75  ,    1]  =  63.4375
  [76  ,    1]  =  24.4444
  [78  ,    1]  =  23.3333
  [123 ,    1]  =  18.5057
  [128 ,    1]  =  53.2743
  [167 ,    1]  =  29.8876
  ⋮
  [5461, 5551]  =  14.4444
  [5468, 5551]  =  34.4416
  [5473, 5551]  =  39.7091
  [5478, 5551]  =  24.96
  [5486, 5551]  =  40.5424
  [5493, 5551]  =  42.4113
  [5498, 5551]  =  24.96
  [5512, 5551]  =  21.4933
  [5518, 5551]  =  16.2029
  [5522, 5551]  =  34.8861
  [5524, 5551]  =  24.4082

The approxSolver is the algorithm that is supposed to run in nearly-linear time. The other solver is the exact solver that we used for the first example above.

In [10]:
# Create a solver and an approximate solver to see performance differences
solver = cholLap(sparseAdj; verbose=true)
approxSolver = approxCholLap(sparseAdj; tol=1e-9, verbose=true, params=ApproxCholParams(:wdeg))

Solver build time: 2.821 seconds.
Solver build time: 3.763 seconds.
Using wted degree ordering. Factorization time: 1.2601959705352783
Ratio of operator edges to original edges: 1.4050065966389584
ratio of max to min diagonal of laplacian : 5669.412534243794
Solver build time: 1.835 seconds.


(::f) (generic function with 1 method)

## The following has been commented out because it took too long to run

## Running the below code gives the following results:
Since the matrix is 5551 x 5551, the code finds the answers to the upper linear systems. This is equivalent to $5550/2 * 5551 = 15,404,025$ or about 15.5 million linear systems with 5551 equations each. Each of the linear systems takes around 0.05 seconds (rounded up) to solve based on the output of running this code for a couple minutes on my computer. If we approximate, that means the code will take ~775,000 seconds to run = ~12,900 minutes = ~215 hours = ~9 days. Keep in mind that this is using my early 2011 MacBook Pro with 4 GB memory. Since this is too long, we will run a shorter sample (just the first row of effective resistance).

```
# Set up effective resistance matrix to hold all of the values
R_effs = zeros(num_users, num_users)

# Loop through to compute the upper values (diagonals will all be 0, lower will be reflection of upper)
for x = 1:num_users-1
    R_effs[x, x+1:num_users] = [compute_effective_resistance(solver, x, y, num_users) for y=x+1:num_users]
end

Symmetric(R_effs)
```

This code computes the first row of the effective resistance matrix with the exact solver since calculating all of the values took too long. Scrolling all the way to the bottom of the output shows the resulting matrix of effective resistances.

In [11]:
# Set up effective resistance matrix to hold all of the values
R_effs = zeros(num_users, num_users)

# Loop through to compute the upper values (diagonals will all be 0, lower will be reflection of upper)
x = 1
@time R_effs[x, x+1:num_users] = [compute_effective_resistance(solver, x, y, num_users) for y=x+1:num_users]

sym_r_eff = Symmetric(R_effs)

Solve time: 0.227 seconds.
Solve time: 0.039 seconds.
Solve time: 0.039 seconds.
Solve time: 0.048 seconds.
Solve time: 0.048 seconds.
Solve time: 0.041 seconds.
Solve time: 0.046 seconds.
Solve time: 0.04 seconds.
Solve time: 0.039 seconds.
Solve time: 0.044 seconds.
Solve time: 0.04 seconds.
Solve time: 0.04 seconds.
Solve time: 0.045 seconds.
Solve time: 0.041 seconds.
Solve time: 0.039 seconds.
Solve time: 0.055 seconds.
Solve time: 0.041 seconds.
Solve time: 0.044 seconds.
Solve time: 0.05 seconds.
Solve time: 0.043 seconds.
Solve time: 0.044 seconds.
Solve time: 0.045 seconds.
Solve time: 0.04 seconds.
Solve time: 0.039 seconds.
Solve time: 0.045 seconds.
Solve time: 0.039 seconds.
Solve time: 0.039 seconds.
Solve time: 0.048 seconds.
Solve time: 0.04 seconds.
Solve time: 0.041 seconds.
Solve time: 0.044 seconds.
Solve time: 0.041 seconds.
Solve time: 0.039 seconds.
Solve time: 0.045 seconds.
Solve time: 0.04 seconds.
Solve time: 0.04 seconds.
Solve time: 0.047 seconds.
Solve tim

Solve time: 0.039 seconds.
Solve time: 0.047 seconds.
Solve time: 0.04 seconds.
Solve time: 0.039 seconds.
Solve time: 0.047 seconds.
Solve time: 0.039 seconds.
Solve time: 0.04 seconds.
Solve time: 0.044 seconds.
Solve time: 0.04 seconds.
Solve time: 0.039 seconds.
Solve time: 0.047 seconds.
Solve time: 0.045 seconds.
Solve time: 0.045 seconds.
Solve time: 0.051 seconds.
Solve time: 0.046 seconds.
Solve time: 0.044 seconds.
Solve time: 0.049 seconds.
Solve time: 0.039 seconds.
Solve time: 0.039 seconds.
Solve time: 0.045 seconds.
Solve time: 0.039 seconds.
Solve time: 0.039 seconds.
Solve time: 0.044 seconds.
Solve time: 0.039 seconds.
Solve time: 0.039 seconds.
Solve time: 0.043 seconds.
Solve time: 0.039 seconds.
Solve time: 0.039 seconds.
Solve time: 0.044 seconds.
Solve time: 0.04 seconds.
Solve time: 0.039 seconds.
Solve time: 0.044 seconds.
Solve time: 0.039 seconds.
Solve time: 0.039 seconds.
Solve time: 0.045 seconds.
Solve time: 0.04 seconds.
Solve time: 0.039 seconds.
Solve 

In [12]:
# Print out the number of components of our generated graph to make sure it is connected
ncomp = 0
for x in components(sparseAdj)
    if x > ncomp
       ncomp = x 
    end
end
ncomp

1

## Show that the exact solver is actually faster / more efficient than the approximate in this case
The solve time of the exact solver is faster so it more than makes up for the solver generation time over multiple calculations. It also gives a more accurate result. The variable b is set up to be a random vector the size of the number of nodes in the network.

In [13]:
b = randn(num_users)
b = b - mean(b)
@time x = solver(b)
la = lap(sparseAdj)
println(norm(la*x - b))

Solve time: 0.158 seconds.
  0.159044 seconds (146 allocations: 374.844 KiB)
1.847321618414873e-13


In [14]:
@time y = approxSolver(b)
println(norm(la*y - b))

PCG Stopped due to small or large rho
PCG stopped after: 0.242 seconds and 11 iterations with relative error 1.2281868750273784e-8.
  0.678651 seconds (158.40 k allocations: 10.637 MiB)
9.059024223143107e-7


# Make a function to go through all of the above steps and find $R_{eff}$

The following functions go through all of the steps in the above calcuations. We then run the functions with the same input to verify that it works.

In [15]:
function find_graph_solver(file_path, num_users)
    println("Reading file info...")
    # Read info from the file
    f = open(file_path, "r");
    lines = readlines(f);
    close(f);
    
    # Create user_citation array assuming same structure as complex example above
    user_citations = [Set{String}(deleteat!(split(l," "), 1)) for l in lines];
    
    # Create sparse adjacency matrix from the user_citations
    Arr = Array{Float64, 2}(num_users, num_users);

    println("Creating sparse adjacency matrix of the graph...")
    # Use indexes to initialize the array
    for user_a = 1:num_users
        P_a = user_citations[user_a]
        for user_b = user_a+1:num_users
            # Since all array values initally zero, we only need to initialize the nonzero values
            P_b = user_citations[user_b]
            if length(intersect(P_a, P_b)) > 0
                s = length(intersect(P_a, P_b))
                Arr[user_a, user_b] = 1 / mean([s/length(P_a), s/length(P_b)])
            end
        end
    end

    # Use the array we have created to make a sparse adjacency matrix of the weighted graph
    sparseAdj = sparse(Symmetric(Arr))
    
    println("Creating a solver for the Laplacian...")
    # Create an exact solver for the given matrix
    @time solver = cholLap(sparseAdj)
end

find_graph_solver (generic function with 1 method)

In [16]:
function matrix_effective_resistance(file_path, num_users, users=[1])
    # Find the solver for our given matrix
    solver = find_graph_solver(file_path, num_users) 

    # Use our solver to populate a matrix with requested values
    # Set up effective resistance matrix to hold all of the values
    R_effs = zeros(num_users, num_users)

    println("Computing effective resistance for the specified users...")
    # Loop through to compute the upper values (diagonals will all be 0, lower will be reflection of upper)
    for x in users
        @time R_effs[x, x+1:num_users] = [compute_effective_resistance(solver, x, y, num_users) for y=x+1:num_users]
    end
        
    return Symmetric(R_effs)
end

matrix_effective_resistance (generic function with 2 methods)

In [17]:
matrix_effective_resistance(file_path, num_users)

Reading file info...
Creating sparse adjacency matrix of the graph...
Creating a solver for the Laplacian...
  3.370489 seconds (455 allocations: 465.185 MiB, 14.29% gc time)
Computing effective resistance for the specified users...
240.743755 seconds (324.65 k allocations: 2.210 GiB, 0.38% gc time)


5551×5551 Symmetric{Float64,Array{Float64,2}}:
 0.0          0.000205194  0.000285793  …  0.000156147  0.000161774
 0.000205194  0.0          0.0             0.0          0.0        
 0.000285793  0.0          0.0             0.0          0.0        
 0.000354298  0.0          0.0             0.0          0.0        
 0.000787306  0.0          0.0             0.0          0.0        
 0.000231772  0.0          0.0          …  0.0          0.0        
 0.000698967  0.0          0.0             0.0          0.0        
 0.000817729  0.0          0.0             0.0          0.0        
 0.00020773   0.0          0.0             0.0          0.0        
 0.000205428  0.0          0.0             0.0          0.0        
 0.000202046  0.0          0.0          …  0.0          0.0        
 0.000120226  0.0          0.0             0.0          0.0        
 8.95147e-5   0.0          0.0             0.0          0.0        
 ⋮                                      ⋱               ⋮          
 

In [18]:
# Run calculations with the citeulike t dataset (it is slightly larger)
citeulike_t = "ctrsr_datasets/citeulike-t/users.dat"
num_t_users = 7947
matrix_effective_resistance(citeulike_t, num_t_users)

Reading file info...
Creating sparse adjacency matrix of the graph...
Creating a solver for the Laplacian...
  3.256036 seconds (521 allocations: 454.870 MiB, 12.81% gc time)
Computing effective resistance for the specified users...
355.134928 seconds (2.27 M allocations: 5.472 GiB, 0.24% gc time)


7947×7947 Symmetric{Float64,Array{Float64,2}}:
 0.0         0.00134212  0.00245161  …  0.00130143  0.00217027  0.00246099
 0.00134212  0.0         0.0            0.0         0.0         0.0       
 0.00245161  0.0         0.0            0.0         0.0         0.0       
 0.0192242   0.0         0.0            0.0         0.0         0.0       
 0.00154903  0.0         0.0            0.0         0.0         0.0       
 0.00395484  0.0         0.0         …  0.0         0.0         0.0       
 0.00149509  0.0         0.0            0.0         0.0         0.0       
 0.00915948  0.0         0.0            0.0         0.0         0.0       
 0.0105719   0.0         0.0            0.0         0.0         0.0       
 0.0359895   0.0         0.0            0.0         0.0         0.0       
 0.00138732  0.0         0.0         …  0.0         0.0         0.0       
 0.00354841  0.0         0.0            0.0         0.0         0.0       
 0.00333695  0.0         0.0            0.0         0

# Create our own weighted adjacency matrix to test solving in a bigger network
The wtedChimera command generates a sparse adjacency matrix of a weighted Chimera graph with 100000 nodes. It is one of the graph generators from the Laplacians library.

In [16]:
num_nodes = 100000
g = wtedChimera(num_nodes)

100000×100000 SparseMatrixCSC{Float64,Int64} with 781756 stored entries:
  [1434  ,      1]  =  2.42915
  [3572  ,      1]  =  2.10177
  [4672  ,      1]  =  1.36739
  [8224  ,      1]  =  2.82197
  [27268 ,      1]  =  0.849172
  [46566 ,      1]  =  1.90639
  [51172 ,      1]  =  1.60818
  [53136 ,      1]  =  2.81398
  [11339 ,      2]  =  2.7729
  [16550 ,      2]  =  2.94963
  ⋮
  [87194 ,  99998]  =  0.496562
  [9877  ,  99999]  =  2.47561
  [75289 ,  99999]  =  1.55287
  [10828 , 100000]  =  1.47279
  [14648 , 100000]  =  2.45439
  [22137 , 100000]  =  2.02958
  [50933 , 100000]  =  1.95538
  [54899 , 100000]  =  1.81257
  [62923 , 100000]  =  3.40848
  [63272 , 100000]  =  1.93296
  [68893 , 100000]  =  2.94271

Once again, loop through the nodes and see that the graph is connected.

In [17]:
x = 0
for i in components(g)
    if i > x
       x = i 
    end
end
x

1

This creates the solver based off of Cholesky factorization and the approximate Cholesky solver again. We can see that the approximate solver is generated much faster.

In [19]:
f = cholLap(g, verbose=true)
app_f = approxCholLap(g, tol=1e-9, verbose=true, params=ApproxCholParams(:wdeg))

Solver build time: 5.379 seconds.
Solver build time: 5.584 seconds.
Using wted degree ordering. Factorization time: 0.7999169826507568
Ratio of operator edges to original edges: 3.170902429914193
ratio of max to min diagonal of laplacian : 194.99363353876078
Solver build time: 1.204 seconds.


(::f) (generic function with 1 method)

Here, we see that solving the system using each of the solvers is much faster with the exact solver. Thus, solving for a significant number of nodes will be faster using the exact solver than the approximate solver. The difference in solving time will eventually make up for the difference of generation time.

In [20]:
@time print(compute_effective_resistance(f, 1, 3140, num_nodes))
@time print(compute_effective_resistance(app_f, 1, 3140, num_nodes))

Solve time: 0.244 seconds.
0.18057867320803223  0.248062 seconds (195 allocations: 6.885 MiB)
PCG Stopped due to small or large rho
PCG stopped after: 0.717 seconds and 26 iterations with relative error 2.4203112232353252e-8.
0.18057867320803211  0.835337 seconds (36.73 k allocations: 67.631 MiB, 15.08% gc time)


## This is a verification that running the full(vec) command is very fast and takes very little memory.

In [25]:
vec = sparsevec(Dict(1=>-1.0, 2=>1.0), 100000)
@time full(vec)

  0.002660 seconds (22 allocations: 782.594 KiB)


100000-element Array{Float64,1}:
 -1.0
  1.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮  
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

## User Interactive Section
In the cells below, create a large graph with one of the [graph generators](http://danspielman.github.io/Laplacians.jl/latest/graphGenerators/index.html#generators) from the Laplacians library. Create an approximate solver and an exact solver and find the effective resistance between any two nodes. See which solver is more efficient for different types of graphs using the `@time` macro. Does this change based on the size of the graph? If you get lost, use the test above as an example.